In [1]:
import re
import sys
import time
sys.path.insert(0, './..')

from openai import OpenAI

from utils import load_dataset, calculate_classification_metric, write_classification_metric

/Users/yukikase/projects/SQLi_detection/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset('./../dataset/test.csv')

In [3]:
model = 'gpt-3.5-turbo'

In [4]:
prompt = [
    {
        'role': 'system',
        'content': '\
            You will be provided with a string. \
            Distinguish if the string causes SQL injection as a web security specialist.\
            If the string can cause SQL injection, return 1. If not, return 0.\
            Return only result for answer. Do not explain the result.'
    },
    {
        'role': 'user',
        'content': '<string>\' or \'a\' = \'a</string>'
    },
    {
        'role': 'assistant',
        'content': '1'
    },
    {
        'role': 'user',
        'content': '<string>Hello.</string>'
    },
    {
        'role': 'assistant',
        'content': '0'
    },
    {
        'role': 'user',
        'content': '<string>I am a Japanese<./string>'
    },
    {
        'role': 'assistant',
        'content': '0'
    },
    {
        'role': 'user',
        'content': '<string>-3756\' )  union all select 1034,1034,1034--</string>'
    },
    {
        'role': 'assistant',
        'content': '1'
    }
]

In [5]:
classification_counts = {
    'true_positives': 0,
    'false_positives': 0,
    'true_negatives': 0,
    'false_negatives': 0
}
inference_time = []
labels = []
raw_queries = []
preds = []

client = OpenAI()

for i, (query, label) in enumerate(dataset[:500]):
    prompt.append(
        {
            'role': 'user',
            'content': f'<string>{query}</string>'
        })

    start_time = time.time()
    chat_completion = client.chat.completions.create(
        model=model,
        messages=prompt,
        temperature=0.1,
        max_tokens=2
    )
    inference_time.append(time.time() - start_time)

    response = chat_completion.choices[0].message.content
    print('response:', response)
    try:
        result = int(re.search(r'(\d+)', response).group(1))
    except ValueError:
        result = 'N/A'
    except AttributeError:
        result = 'N/A'

    labels.append(int(label))
    raw_queries.append(query)
    preds.append(result)


    print(i, result, label)
    prompt.pop(9)

calculate_classification_metric(classification_counts, labels, preds, raw_queries)
write_classification_metric(classification_counts, inference_time)

response: 0
0 0 0
response: 1
1 1 1
response: 1
2 1 1
response: 0
3 0 0
response: 0
4 0 0
response: 0
5 0 0
response: 1
6 1 1
response: 0
7 0 0
response: 1
8 1 1
response: 0
9 0 0
response: 1
10 1 1
response: 0
11 0 0
response: 0
12 0 0
response: 0
13 0 0
response: 0
14 0 0
response: 1
15 1 1
response: 0
16 0 0
response: 1
17 1 1
response: 0
18 0 0
response: 0
19 0 0
response: 0
20 0 0
response: 0
21 0 0
response: 1
22 1 1
response: 0
23 0 0
response: 1
24 1 1
response: 1
25 1 1
response: 0
26 0 0
response: 1
27 1 1
response: 1
28 1 1
response: 1
29 1 1
response: 0
30 0 0
response: 0
31 0 0
response: 0
32 0 0
response: 1
33 1 1
response: 1
34 1 1
response: 0
35 0 0
response: 0
36 0 0
response: 0
37 0 0
response: 1
38 1 1
response: 1
39 1 1
response: 0
40 0 0
response: 0
41 0 0
response: 0
42 0 0
response: 0
43 0 0
response: 0
44 0 0
response: 1
45 1 1
response: 0
46 0 0
response: 1
47 1 1
response: 1
48 1 1
response: 1
49 1 1
response: 0
50 0 0
response: 0
51 0 0
response: 0
52 0 0
res